#### **MODELING**
This notebook implements machine learning models based on the comprehensive EDA findings and preprocessing recommendations. We follow an evidence-based approach to ensure the modeling strategy aligns with the data characteristics and patterns identified earlier.

**Modeling Strategy**

1. Model Selection – Logistic Regression (baseline), Random Forest, and XGBoost (EDA recommendations)

2. Model Evaluation – Confusion matrix, classification report, ROC-AUC, balanced accuracy, and macro F1
3. Explainability – SHAP values and model coefficients for feature interpretation

**EDA Evidence for Modeling**
1. Non-linear interactions – Use tree-based models (Random Forest, XGBoost) to capture complex feature relationships

2. Production focus – Select interpretable and stable models suitable for deployment and policy insights

#### **1. Import Libraries and Load Preprocessed Data**

In [8]:
#Core Libaries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

# Machine learning libaries
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import (classification_report,confusion_matrix, accuracy_score,balanced_accuracy_score,f1_score,precision_score,recall_score)
from sklearn.preprocessing import StandardScaler
import joblib

#Advanced ML Libraries
try:
    import xgboost as xgb
    XGBOOST_AVAILABLE = True
except ImportError:
    XGBOOST_AVAILABLE = False
    print("XGBoost not available. Install with: pip install xgboost")

try:
    import lightgbm as lgb
    LIGHTGBM_AVAILABLE = True
except ImportError:
    LIGHTGBM_AVAILABLE = False
    print("LightGBM not available. Install with: pip install lightgbm")

# Explainability
try:
    import shap
    SHAP_AVAILABLE = True
except ImportError:
    SHAP_AVAILABLE = False
    print("SHAP not available. Install with: pip install shap")

# Set style for better visualizations
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully!")
print(f"XGBoost available: {XGBOOST_AVAILABLE}")
print(f"LightGBM available: {LIGHTGBM_AVAILABLE}")
print(f"SHAP available: {SHAP_AVAILABLE}")

Libraries imported successfully!
XGBoost available: True
LightGBM available: True
SHAP available: True


In [9]:
#load preprocessed datasets
print("LOADING PREPROCESSED DATA")

try:
    # load preprocessed datasets
    X_train = pd.read_csv("scaled_X_train.csv")
    X_test = pd.read_csv("scaled_X_test.csv")

    y_train = pd.read_csv("y_train.csv").squeeze()
    y_test = pd.read_csv("y_test.csv").squeeze()
     ### The .squeeze() method removes dimensions of size 1, converting a DataFrame to a Series
    ### The .squeeze() method is a clean, robust way to ensure your target variables are in the correct 1D format that sklearn expects, 
    ### preventing potential errors during model training and evaluation.

    #Load preprocessing objects
    scaler = joblib.load("scaler.pkl")
    selected_features = joblib.load("selected_features.pkl")

    print("...Preprocessed data loaded successfully!!....")
    print(f"Training set: {X_train.shape}")
    print(f"Test set: {X_test.shape}")

      # Display class distribution
    print("\nClass distribution:")
    print("Training set:")
    print(y_train.value_counts().sort_index())
    print("\nValidation set:")
    print("\nTest set:")
    print(y_test.value_counts().sort_index())

     # Display selected features
    print(f"\nSelected features ({len(selected_features)}):")
    for i, feature in enumerate(selected_features, 1):
        print(f"{i:2d}. {feature}")


except FileNotFoundError as e:
    print(f"Error loading preprocessed data: {e}")
    print("Please run the preprocessing notebook first to generate the required files.")
    print("Required files: X_train_scaled.csv,  X_test_scaled.csv")
    print("\ny_train.csv,y_test.csv")
    print("\nscaler.pkl, selected_features.pkl")

    




LOADING PREPROCESSED DATA
...Preprocessed data loaded successfully!!....
Training set: (675, 15)
Test set: (169, 15)

Class distribution:
Training set:
Loan_Status
0    328
1    347
Name: count, dtype: int64

Validation set:

Test set:
Loan_Status
0    94
1    75
Name: count, dtype: int64

Selected features (15):
 1. Gender
 2. Married
 3. Dependents
 4. Education
 5. Self_Employed
 6. Credit_History
 7. Property_Area
 8. Loan_Amount_Term_ in_year
 9. Income_Category
10. Has_Coapplicant
11. ApplicantIncome_log
12. CoapplicantIncome_log
13. LoanAmount_log
14. Total_income_log
15. EMI_log


#### **2. Baseline Model - Logistic Regression**

In [ ]:
# Baseline Logistic Regression Model
print("BASELINE LOGISTIC REGRESSION MODEL")
print("EDA recommendation: Fast, interpretable baseline with coefficients for feature effect direction")


 # create and train baseline model
baseline_model = LogisticRegression(
    random_state=234,
    max_iter=1000)
# Train model
baseline_model.fit(X_train, y_train)

# Make predictions
y_train_pred = baseline_model.predict(X_train)
y_test_pred = baseline_model.predict(X_test)

# Calculate metrics
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

train_f1 = f1_score(y_train, y_train_pred, average='binary')
test_f1 = f1_score(y_test, y_test_pred, average='binary')

print(f"\nBaseline Model Performance:")
print(f"Training - Accuracy: {train_accuracy:.3f}, , Binary F1: {train_f1:.3f}")

print(f"Test - Accuracy: {test_accuracy:.3f}, , Binary F1: {test_f1:.3f}")


# Display feature coefficients (interpretability)
print(f"\nFeature Coefficients (Top 10):")
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'coefficient': baseline_model.coef_[0]
}).sort_values('coefficient', key=abs, ascending=False)

for i, (_, row) in enumerate(feature_importance.head(10).iterrows(), 1):
    print(f"{i:2d}. {row['feature']}: {row['coefficient']:.3f}")

# Store baseline results
baseline_results = {
    'model': 'Logistic Regression',
    'train_accuracy': train_accuracy,
    'test_accuracy': test_accuracy,
    'test_f1': test_f1
}

print(".... Baseline model completed!")



BASELINE LOGISTIC REGRESSION MODEL
EDA recommendation: Fast, interpretable baseline with coefficients for feature effect direction

Baseline Model Performance:
Training - Accuracy: 0.776, , Macro F1: 0.805
Test - Accuracy: 0.716, , Macro F1: 0.742

Feature Coefficients (Top 10):
 1. Married: 1.340
 2. Credit_History: 1.216
 3. Self_Employed: 1.017
 4. CoapplicantIncome_log: 0.875
 5. Has_Coapplicant: -0.649
 6. Income_Category: 0.564
 7. Property_Area: 0.531
 8. Total_income_log: -0.468
 9. ApplicantIncome_log: 0.423
10. EMI_log: -0.417
.... Baseline model completed!
